<a href="https://colab.research.google.com/github/ndlovuntando/tensorflow_dev_certificate/blob/main/imdb_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np